Carregando o CSV com os dados dos alunos referentes à 2016

import os
import pandas as pd
import numpy as np3

if not os.path.exists("data"):
    os.mkdir("data")
    !cd data && curl -O {"http://download.inep.gov.br/microdados/microdados_censo_superior_2016.zip"} && unzip {"'*.zip'"}
    !cd data/DADOS && unrar e "*.rar" && mv *.CSV ../
    !cd data && rm -r ANEXOS LEIA-ME DADOS FILTROS && rm microdados_censo_superior_2016.zip



Coloquei em markdown porque tava demorando pra baixar, já que eu tinha os arquivos aqui... preferi deixar assim por enquanto

import pandas as pd

dataframe_alunos_2016 =  pd.read_csv('data/DM_ALUNO.CSV',
                                     sep='|', encoding='latin1',
                                     usecols=['CO_IES','NO_IES' ,'CO_TURNO_ALUNO', 'CO_CATEGORIA_ADMINISTRATIVA','CO_GRAU_ACADEMICO', 'CO_ALUNO_CURSO', 'DS_COR_RACA_ALUNO',
                                            'CO_MODALIDADE_ENSINO', 'NO_OCDE', 'DS_SEXO_ALUNO', 'NU_ANO_ALUNO_NASC', 'CO_ALUNO_SITUACAO',
                                             'IN_FINANC_ESTUDANTIL', 'CO_TIPO_ESCOLA_ENS_MEDIO', 'ANO_INGRESSO', 'IN_DEF_FISICA'])
dataframe_alunos_2016.head()

print(dataframe_alunos_2016.groupby('IN_FINANC_ESTUDANTIL').count())

dataframe_alunos_2016.to_csv('data/alunos2016.csv')

In [205]:
import pandas as pd

import numpy as np

DF_alunos2016 = dataframe_alunos_2016  #pd.read_csv('data/alunos2016.csv')
DF_alunos2016.head()

,CO_IES,NO_IES,CO_CATEGORIA_ADMINISTRATIVA,CO_TURNO_ALUNO,CO_GRAU_ACADEMICO,CO_MODALIDADE_ENSINO,NO_OCDE,CO_ALUNO_CURSO,DS_COR_RACA_ALUNO,DS_SEXO_ALUNO,NU_ANO_ALUNO_NASC,IN_DEF_FISICA,CO_ALUNO_SITUACAO,IN_FINANC_ESTUDANTIL,CO_TIPO_ESCOLA_ENS_MEDIO,ANO_INGRESSO
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1,3.0,1.0,1,Direito,25001314,Parda,Masculino,1987,NaN,2,NaN,1,2016
1,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1,3.0,1.0,1,Direito,25037243,Parda,Masculino,1988,NaN,2,NaN,1,2012
2,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1,3.0,1.0,1,Direito,25030916,Preta,Feminino,1988,NaN,3,NaN,1,2015
3,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1,3.0,1.0,1,Direito,25016919,Parda,Masculino,1977,NaN,3,NaN,1,2013
4,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1,1.0,1.0,1,Direito,25001942,Parda,Masculino,1983,NaN,4,NaN,1,2011


In [206]:
qtd_cursos = len(DF_alunos2016.NO_OCDE.drop_duplicates())
qtd_cursos

350

In [207]:
curso_raca_Publico = DF_alunos2016[((DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 1)|(DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 2)|
                                  (DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 3))]

curso_raca_Publico = pd.melt(curso_raca_Publico, id_vars = 'NO_OCDE', value_vars ='DS_COR_RACA_ALUNO')

curso_raca_Publico.head()



,NO_OCDE,variable,value
0,Direito,DS_COR_RACA_ALUNO,Parda
1,Direito,DS_COR_RACA_ALUNO,Parda
2,Direito,DS_COR_RACA_ALUNO,Preta
3,Direito,DS_COR_RACA_ALUNO,Parda
4,Direito,DS_COR_RACA_ALUNO,Parda


In [208]:
curso_raca_Privado = DF_alunos2016[((DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 4)|(DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 5))]

curso_raca_Privado = pd.melt(curso_raca_Privado, id_vars = 'NO_OCDE', value_vars ='DS_COR_RACA_ALUNO')

curso_raca_Privado.head()

,NO_OCDE,variable,value
0,Formação de professor de filosofia,DS_COR_RACA_ALUNO,Preta
1,Formação de professor de filosofia,DS_COR_RACA_ALUNO,Branca
2,Formação de professor de filosofia,DS_COR_RACA_ALUNO,Aluno não quis declarar cor/raça
3,Formação de professor de filosofia,DS_COR_RACA_ALUNO,Branca
4,Formação de professor de filosofia,DS_COR_RACA_ALUNO,Aluno não quis declarar cor/raça


In [209]:
curso_raca_Publico = curso_raca_Publico.rename(columns= {'value' : 'raca'})
curso_raca_Publico = curso_raca_Publico.rename(columns= {'NO_OCDE' : 'curso'})
curso_raca_Publico = curso_raca_Publico.rename(columns= {'variable' : 'qtd'})

curso_raca_Publico = curso_raca_Publico.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

curso_raca_Publico = curso_raca_Publico.pivot_table(index = 'curso', columns='raca', values= 'qtd',aggfunc='sum')

curso_raca_Publico.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
curso,,,,,,,
Acústica,4.0,NaN,187.0,NaN,NaN,19.0,10.0
Administração,26373.0,1435.0,36738.0,452.0,3531.0,25066.0,6898.0
Administração de cooperativas,246.0,16.0,375.0,6.0,3.0,332.0,140.0
Administração de redes,696.0,39.0,1510.0,7.0,158.0,1179.0,337.0
Administração dos serviços de saúde,698.0,29.0,676.0,39.0,49.0,712.0,218.0


In [210]:
curso_raca_Privado = curso_raca_Privado.rename(columns= {'value' : 'raca'})
curso_raca_Privado = curso_raca_Privado.rename(columns= {'NO_OCDE' : 'curso'})
curso_raca_Privado = curso_raca_Privado.rename(columns= {'variable' : 'qtd'})

curso_raca_Privado = curso_raca_Privado.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

curso_raca_Privado = curso_raca_Privado.pivot_table(index = 'curso', columns='raca', values= 'qtd',aggfunc='sum')

curso_raca_Privado.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
curso,,,,,,,
Administração,318098.0,13671.0,351847.0,4318.0,31094.0,218476.0,47193.0
Administração de cooperativas,152.0,1.0,222.0,NaN,3.0,50.0,16.0
Administração de redes,10598.0,536.0,9767.0,191.0,766.0,7739.0,2281.0
Administração de sistemas de informações,NaN,NaN,NaN,NaN,NaN,6.0,NaN
Administração dos serviços de saúde,139.0,5.0,162.0,1.0,NaN,72.0,13.0


In [211]:
curso_raca_Publico.loc['Administração']

raca
Aluno não quis declarar cor/raça    26373.0
Amarela                              1435.0
Branca                              36738.0
Indígena                              452.0
Não dispõe da informação             3531.0
Parda                               25066.0
Preta                                6898.0
Name: Administração, dtype: float64

Com isso, podemos selecionar relações entre os cursos e as raças

In [212]:
for i in range(0,6):
    print(curso_raca_Publico.iloc[:,i].idxmax())



Pedagogia
Pedagogia
Pedagogia
Licenciatura Intercultural Indígena
Pedagogia
Pedagogia


Pegando o curso com maior qtd por coluna

In [213]:
curso_raca_Publico.iloc[:].nlargest(5,'Amarela')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
curso,,,,,,,
Pedagogia,42901.0,1543.0,38189.0,1396.0,6822.0,42480.0,13311.0
Administração,26373.0,1435.0,36738.0,452.0,3531.0,25066.0,6898.0
Medicina,9749.0,1143.0,23103.0,339.0,1349.0,11599.0,3080.0
Direito,20676.0,915.0,32749.0,546.0,2818.0,20620.0,6687.0
Formação de professor de matemática,20404.0,892.0,20531.0,342.0,2315.0,24668.0,5803.0


Dessa forma mostra os n maiores cursos por raça, mas "ordena" pela coluna selecionada, nesse caso, foi "Amarela"

In [214]:
curso_raca_Publico.fillna(0,inplace = True)
curso_raca_Privado.fillna(0,inplace = True)

In [215]:
curso_raca_Publico['total'] = (curso_raca_Publico['Amarela'] + curso_raca_Publico['Aluno não quis declarar cor/raça'] + curso_raca_Publico['Branca'] 
                               + curso_raca_Publico['Indígena'] +  curso_raca_Publico['Parda'] + curso_raca_Publico['Preta'] + curso_raca_Publico['Não dispõe da informação'] )

curso_raca_Privado['total'] = (curso_raca_Privado['Amarela'] + curso_raca_Privado['Aluno não quis declarar cor/raça'] + curso_raca_Privado['Branca'] 
                               + curso_raca_Privado['Indígena'] + curso_raca_Privado['Parda'] + curso_raca_Privado['Preta'] + curso_raca_Privado['Não dispõe da informação'] )


In [216]:
curso_raca_Publico['% Branca'] =np.round((curso_raca_Publico['Branca'] / curso_raca_Publico['total']) *100, decimals=2)
curso_raca_Publico['% Amarela'] = np.round((curso_raca_Publico['Amarela'] / curso_raca_Publico['total']) * 100, decimals=2)
curso_raca_Publico['% Indígena'] = np.round((curso_raca_Publico['Indígena'] / curso_raca_Publico['total']) *100, decimals=2)
curso_raca_Publico['% Parda'] = np.round((curso_raca_Publico['Parda'] / curso_raca_Publico['total']) *100, decimals=2)
curso_raca_Publico['% Preta'] = np.round((curso_raca_Publico['Preta'] / curso_raca_Publico['total']) *100, decimals=2)
curso_raca_Publico['% Aluno não quis declarar cor/raça'] = np.round((curso_raca_Publico['Aluno não quis declarar cor/raça'] / curso_raca_Publico['total']) *100,decimals=2)
curso_raca_Publico['% Não dispõe da informação'] = np.round((curso_raca_Publico['Não dispõe da informação'] / curso_raca_Publico['total']) *100,decimals=2)


curso_raca_Privado['% Branca'] =np.round((curso_raca_Privado['Branca'] / curso_raca_Privado['total']) *100, decimals=2)
curso_raca_Privado['% Amarela'] = np.round((curso_raca_Privado['Amarela'] / curso_raca_Privado['total']) * 100, decimals=2)
curso_raca_Privado['% Indígena'] = np.round((curso_raca_Privado['Indígena'] / curso_raca_Privado['total']) *100, decimals=2)
curso_raca_Privado['% Parda'] = np.round((curso_raca_Privado['Parda'] / curso_raca_Privado['total']) *100, decimals=2)
curso_raca_Privado['% Preta'] = np.round((curso_raca_Privado['Preta'] / curso_raca_Privado['total']) *100, decimals=2)
curso_raca_Privado['% Aluno não quis declarar cor/raça'] = np.round((curso_raca_Privado['Aluno não quis declarar cor/raça'] / curso_raca_Privado['total']) *100,decimals=2)
curso_raca_Privado['% Não dispõe da informação'] = np.round((curso_raca_Privado['Não dispõe da informação'] / curso_raca_Privado['total']) *100,decimals=2)


In [217]:
curso_raca_Publico.iloc[:].nlargest(5,'total')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta,total,% Branca,% Amarela,% Indígena,% Parda,% Preta,% Aluno não quis declarar cor/raça,% Não dispõe da informação
curso,,,,,,,,,,,,,,,
Pedagogia,42901.0,1543.0,38189.0,1396.0,6822.0,42480.0,13311.0,146642.0,26.04,1.05,0.95,28.97,9.08,29.26,4.65
Administração,26373.0,1435.0,36738.0,452.0,3531.0,25066.0,6898.0,100493.0,36.56,1.43,0.45,24.94,6.86,26.24,3.51
Direito,20676.0,915.0,32749.0,546.0,2818.0,20620.0,6687.0,85011.0,38.52,1.08,0.64,24.26,7.87,24.32,3.31
Formação de professor de matemática,20404.0,892.0,20531.0,342.0,2315.0,24668.0,5803.0,74955.0,27.39,1.19,0.46,32.91,7.74,27.22,3.09
Formação de professor de biologia,18773.0,839.0,20697.0,353.0,3154.0,20781.0,5900.0,70497.0,29.36,1.19,0.50,29.48,8.37,26.63,4.47


In [218]:
curso_raca_Privado.iloc[:].nlargest(5,'total')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta,total,% Branca,% Amarela,% Indígena,% Parda,% Preta,% Aluno não quis declarar cor/raça,% Não dispõe da informação
curso,,,,,,,,,,,,,,,
Direito,286339.0,16112.0,414446.0,5861.0,20614.0,234826.0,55133.0,1033331.0,40.11,1.56,0.57,22.73,5.34,27.71,1.99
Administração,318098.0,13671.0,351847.0,4318.0,31094.0,218476.0,47193.0,984697.0,35.73,1.39,0.44,22.19,4.79,32.30,3.16
Pedagogia,249834.0,8469.0,252705.0,4390.0,26626.0,177443.0,39686.0,759153.0,33.29,1.12,0.58,23.37,5.23,32.91,3.51
Ciências contábeis,152356.0,6701.0,152717.0,2381.0,9799.0,110862.0,22815.0,457631.0,33.37,1.46,0.52,24.23,4.99,33.29,2.14
Engenharia civil,122791.0,8584.0,175045.0,3366.0,6322.0,98270.0,22761.0,437139.0,40.04,1.96,0.77,22.48,5.21,28.09,1.45


In [219]:
DF_IES =  pd.read_csv('data/DM_IES.CSV', sep='|', encoding='latin1')
DF_IES.head()

,CO_IES,NO_IES,SGL_IES,CO_MANTENEDORA,NO_MANTENEDORA,CO_CATEGORIA_ADMINISTRATIVA,DS_CATEGORIA_ADMINISTRATIVA,CO_ORGANIZACAO_ACADEMICA,DS_ORGANIZACAO_ACADEMICA,CO_MUNICIPIO_IES,...,VL_RECEITA_PROPRIA,VL_TRANSFERENCIA,VL_OUTRA_RECEITA,VL_DES_PESSOAL_REM_DOCENTE,VL_DES_PESSOAL_REM_TECNICO,VL_DES_PESSOAL_ENCARGO,VL_DES_CUSTEIO,VL_DES_INVESTIMENTO,VL_DES_PESQUISA,VL_DES_OUTRAS
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,1,FUNDACAO UNIVERSIDADE FEDERAL DE MATO GROSSO,1,Pública Federal,1,Universidade,5103403,...,7847404.77,8.463274e+08,0.000000e+00,2.409985e+08,1.047201e+08,1.016396e+08,1.301580e+08,17926971.06,14034108.54,2.446976e+08
1,2,UNIVERSIDADE DE BRASÍLIA,UNB,2,FUNDACAO UNIVERSIDADE DE BRASILIA,1,Pública Federal,1,Universidade,5300108,...,42952090.56,1.125740e+08,1.270250e+08,9.406867e+08,0.000000e+00,9.406867e+08,3.031958e+08,29754378.97,12466559.85,3.734387e+08
2,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,3,FUNDACAO UNIVERSIDADE FEDERAL DE SERGIPE,1,Pública Federal,1,Universidade,2806701,...,4744562.76,7.149826e+08,2.693960e+08,2.817225e+08,1.516967e+08,1.128631e+08,1.558269e+08,53033112.85,25766649.67,0.000000e+00
3,4,UNIVERSIDADE FEDERAL DO AMAZONAS,UFAM,4,FUNDACAO UNIVERSIDADE DO AMAZONAS,1,Pública Federal,1,Universidade,1302603,...,7331203.25,6.319321e+08,7.760058e+05,3.123510e+08,0.000000e+00,9.795765e+07,1.474634e+08,31936025.81,1055524.04,1.368965e+08
4,6,UNIVERSIDADE FEDERAL DE OURO PRETO,UFOP,6,UNIVERSIDADE FEDERAL DE OURO PRETO,1,Pública Federal,1,Universidade,3146107,...,3746358.40,4.009708e+08,3.957849e+06,1.543418e+08,7.137004e+07,3.582467e+07,4.110826e+07,1676309.58,1142377.41,1.032126e+08


In [220]:
IES_raca_Privado = DF_alunos2016[((DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 4)|(DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 5))]
IES_raca_Privado = pd.melt(IES_raca_Privado, id_vars = 'NO_IES', value_vars ='DS_COR_RACA_ALUNO')

IES_raca_Privado.head()


,NO_IES,variable,value
0,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,DS_COR_RACA_ALUNO,Preta
1,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,DS_COR_RACA_ALUNO,Branca
2,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,DS_COR_RACA_ALUNO,Aluno não quis declarar cor/raça
3,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,DS_COR_RACA_ALUNO,Branca
4,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,DS_COR_RACA_ALUNO,Aluno não quis declarar cor/raça


In [221]:
IES_raca_Publico = DF_alunos2016[((DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 1)|(DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 2)
                                 |(DF_alunos2016.CO_CATEGORIA_ADMINISTRATIVA == 3) )]
IES_raca_Publico = pd.melt(IES_raca_Publico, id_vars = 'NO_IES', value_vars ='DS_COR_RACA_ALUNO')

IES_raca_Publico.head()


,NO_IES,variable,value
0,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
1,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
2,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Preta
3,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
4,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda


In [222]:
IES_raca_Publico = IES_raca_Publico.rename(columns= {'value' : 'raca'})
IES_raca_Publico = IES_raca_Publico.rename(columns= {'NO_IES' : 'IES'})
IES_raca_Publico = IES_raca_Publico.rename(columns= {'variable' : 'qtd'})

IES_raca_Publico = IES_raca_Publico.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

IES_raca_Publico = IES_raca_Publico.pivot_table(index = 'IES', columns='raca', values= 'qtd',aggfunc='sum')

IES_raca_Publico.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
IES,,,,,,,
CENTRO DE EDUCAÇÃO TÉCNICA DA UTRAMIG,35.0,NaN,9.0,1.0,NaN,15.0,1.0
CENTRO DE ENSINO SUPERIOR DE ARCOVERDE,954.0,3.0,63.0,8.0,172.0,146.0,22.0
CENTRO DE ENSINO SUPERIOR DO VALE SÃO FRANCISCO,93.0,8.0,51.0,29.0,300.0,417.0,88.0
CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO SUCKOW DA FONSECA,5476.0,16.0,802.0,8.0,1.0,1121.0,167.0
CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA DE MINAS GERAIS,721.0,183.0,2391.0,11.0,390.0,1811.0,354.0


In [223]:
IES_raca_Privado = IES_raca_Privado.rename(columns= {'value' : 'raca'})
IES_raca_Privado = IES_raca_Privado.rename(columns= {'NO_IES' : 'IES'})
IES_raca_Privado = IES_raca_Privado.rename(columns= {'variable' : 'qtd'})

IES_raca_Privado = IES_raca_Privado.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

IES_raca_Privado = IES_raca_Privado.pivot_table(index = 'IES', columns='raca', values= 'qtd',aggfunc='sum')

IES_raca_Privado.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
IES,,,,,,,
ABEU - CENTRO UNIVERSITÁRIO,979.0,83.0,1822.0,18.0,440.0,2255.0,731.0
ALFA - Faculdade de Almenara,421.0,1.0,18.0,1.0,22.0,48.0,2.0
CASTELLI ESCOLA SUPERIOR DE HOTELARIA,43.0,1.0,19.0,NaN,1.0,3.0,NaN
CENTRO DE ENSINO SUPERIOR ARCANJO MIKAEL DE ARAPIRACA,140.0,2.0,159.0,NaN,NaN,185.0,5.0
CENTRO DE ENSINO SUPERIOR DE CATALÃO,14.0,38.0,769.0,1.0,24.0,449.0,64.0


In [224]:
IES_raca_Publico.fillna(0,inplace = True)

IES_raca_Privado.fillna(0,inplace = True)

In [225]:
IES_raca_Publico['total'] = (IES_raca_Publico['Amarela'] + IES_raca_Publico['Aluno não quis declarar cor/raça'] + IES_raca_Publico['Branca'] + IES_raca_Publico['Indígena']
                    +  IES_raca_Publico['Parda'] + IES_raca_Publico['Preta'] + IES_raca_Publico['Não dispõe da informação'] )



IES_raca_Privado['total'] = (IES_raca_Privado['Amarela'] + IES_raca_Privado['Aluno não quis declarar cor/raça'] + IES_raca_Privado['Branca'] + IES_raca_Privado['Indígena']
                    +  IES_raca_Privado['Parda'] + IES_raca_Privado['Preta'] + IES_raca_Privado['Não dispõe da informação'] )
                    
                    

In [226]:
IES_raca_Publico['% Branca'] =np.round((IES_raca_Publico['Branca'] / IES_raca_Publico['total']) *100, decimals=2)
IES_raca_Publico['% Amarela'] = np.round((IES_raca_Publico['Amarela'] / IES_raca_Publico['total']) * 100, decimals=2)
IES_raca_Publico['% Indígena'] = np.round((IES_raca_Publico['Indígena'] / IES_raca_Publico['total']) *100, decimals=2)
IES_raca_Publico['% Parda'] = np.round((IES_raca_Publico['Parda'] / IES_raca_Publico['total']) *100, decimals=2)
IES_raca_Publico['% Preta'] = np.round((IES_raca_Publico['Preta'] / IES_raca_Publico['total']) *100, decimals=2)
IES_raca_Publico['% Aluno não quis declarar cor/raça'] = np.round((IES_raca_Publico['Aluno não quis declarar cor/raça'] / IES_raca_Publico['total']) *100,decimals=2)
IES_raca_Publico['% Não dispõe da informação'] = np.round((IES_raca_Publico['Não dispõe da informação'] / IES_raca_Publico['total']) *100,decimals=2)


IES_raca_Privado['% Branca'] =np.round((IES_raca_Privado['Branca'] / IES_raca_Privado['total']) *100, decimals=2)
IES_raca_Privado['% Amarela'] = np.round((IES_raca_Privado['Amarela'] / IES_raca_Privado['total']) * 100, decimals=2)
IES_raca_Privado['% Indígena'] = np.round((IES_raca_Privado['Indígena'] / IES_raca_Privado['total']) *100, decimals=2)
IES_raca_Privado['% Parda'] = np.round((IES_raca_Privado['Parda'] / IES_raca_Privado['total']) *100, decimals=2)
IES_raca_Privado['% Preta'] = np.round((IES_raca_Privado['Preta'] / IES_raca_Privado['total']) *100, decimals=2)
IES_raca_Privado['% Aluno não quis declarar cor/raça'] = np.round((IES_raca_Privado['Aluno não quis declarar cor/raça'] / IES_raca_Privado['total']) *100,decimals=2)
IES_raca_Privado['% Não dispõe da informação'] = np.round((IES_raca_Privado['Não dispõe da informação'] / IES_raca_Privado['total']) *100,decimals=2)


In [227]:
IES_raca_Publico.iloc[:].nlargest(5,'% Branca')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta,total,% Branca,% Amarela,% Indígena,% Parda,% Preta,% Aluno não quis declarar cor/raça,% Não dispõe da informação
IES,,,,,,,,,,,,,,,
CENTRO UNIVERSITÁRIO DE UNIÃO DA VITÓRIA,18.0,0.0,1446.0,0.0,43.0,4.0,0.0,1511.0,95.70,0.00,0.00,0.26,0.00,1.19,2.85
FACULDADE DE MEDICINA DE JUNDIAÍ,1.0,41.0,468.0,0.0,0.0,1.0,1.0,512.0,91.41,8.01,0.00,0.20,0.20,0.20,0.00
FACULDADES INTEGRADAS REGIONAIS DE AVARÉ,28.0,1.0,663.0,2.0,2.0,43.0,7.0,746.0,88.87,0.13,0.27,5.76,0.94,3.75,0.27
FACULDADE DE MEDICINA DE SÃO JOSÉ DO RIO PRETO,15.0,28.0,493.0,0.0,1.0,41.0,11.0,589.0,83.70,4.75,0.00,6.96,1.87,2.55,0.17
UNIVERSIDADE FEDERAL DE SANTA CATARINA,140.0,440.0,29553.0,140.0,51.0,3519.0,1823.0,35666.0,82.86,1.23,0.39,9.87,5.11,0.39,0.14


In [228]:
IES_raca_Publico.loc['UNIVERSIDADE DE SÃO PAULO']

raca
Aluno não quis declarar cor/raça      18026.00
Amarela                                3604.00
Branca                                41553.00
Indígena                                106.00
Não dispõe da informação                  3.00
Parda                                  5325.00
Preta                                  1482.00
total                                 70099.00
% Branca                                 59.28
% Amarela                                 5.14
% Indígena                                0.15
% Parda                                   7.60
% Preta                                   2.11
% Aluno não quis declarar cor/raça       25.72
% Não dispõe da informação                0.00
Name: UNIVERSIDADE DE SÃO PAULO, dtype: float64

In [229]:
DF_alunos2016.isnull().sum()

CO_IES                                0
NO_IES                                0
CO_CATEGORIA_ADMINISTRATIVA           0
CO_TURNO_ALUNO                  2350313
CO_GRAU_ACADEMICO                 60766
CO_MODALIDADE_ENSINO                  0
NO_OCDE                           52270
CO_ALUNO_CURSO                        0
DS_COR_RACA_ALUNO                     0
DS_SEXO_ALUNO                         0
NU_ANO_ALUNO_NASC                     0
IN_DEF_FISICA                  11399409
CO_ALUNO_SITUACAO                     0
IN_FINANC_ESTUDANTIL            2232187
CO_TIPO_ESCOLA_ENS_MEDIO              0
ANO_INGRESSO                          0
dtype: int64

In [230]:
defFisica = DF_alunos2016

defFisica = defFisica.dropna(subset=['IN_DEF_FISICA'])